### Item 51: Define a Root Exception to Insulate Callers from APIs

* When you're defining a module's API, the exceptions you throw are just as much a part of your interface as the functions and classes you define.
    * See `Item 14`: Prefer Exceptions to Returning None.


* Python has a built-in hierarchy of exceptions for the language and standard library.

In [ ]:
import logging

import my_module

In [ ]:
dir(my_module)

In [ ]:
def determine_weight(volume, density):
    if density <= 0:
        raise ValueError("Density must be positive")
    # ...

* In some cases, using ValueError makes sense, but for APIs it's much more powerful to define your own hierarchyof exceptions.
* You can do this by providing a root Exception in your module.
* Then, have all other exceptions raised by that module inherit from the root exception.

In [ ]:
# # my_module.py
class Error(Exception):
    """Base-class for all exceptions raised by this module."""


class InvalidDensityError(Error):
    """There was a problem with a provided density value."""

* Having a root exception in a module makes it easy for consumers of your API to catch all of the exceptions that you raise on purpose.

In [ ]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.Error as e:
    logging.error(f"Unexpected error: {e!r}")

* This `try/except` prevents your API's exceptions from propagating too far upward and breaking the calling program.
* It insulates the calling code from your API.
* This insulation has three helpful effects.
    1. `root exceptions` let callers understand when there's a problem with their usage of your API.
        * If callers are using your API properly, they should catch the various exceptions that you deliberately raise.
        * If they don't handle such an exception, it will propagate all the way up to the insulatin gexcept block that catches your module's root exception.
   

In [ ]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error(f"Bug in the calling code: {e!r}")

 2. They can help find bugs in your API module's code.

In [ ]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error(f"Bug in the calling code: {e!r}")

3. Future-proofing your API.
    * Over time, you may want to expand your API to provide more specific exceptions in certain situation.

In [ ]:
# my_module.py
class NegativeDensityError(InvalidDensityError):
    """A provided density value was negative"""

In [ ]:
def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError

* The calling code will continue to work exactly as before because it already catches InvalidDensityError exceptions (the parent class of NegativeDensityError).
* In the future, the caller could decide to special-case the new type of exception and change its behavior accordingly:

In [ ]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.NegativeDensityError as e:
    raise ValueError("Must supply non-negative density") from e
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error(f"Bug in the calling code: {e!r}")
except Exception as e:
    logging.error(f"Bug in the API code: {e!r}")
    raise

* You can take API future-proofing further by providing a broader set of exceptions directly below the root exception.
* For example, imagine you had one set of errors related to calculating weights, another related to calculating volume, and a third related to calculating density.

In [ ]:
# my_module.py
class WeightError(Error):
    """Base-class for weight calculation errors."""
    
    
class VolumeError(Error):
    """Base-class for volume calculation errors."""
    
    
class DensityError(Error):
    """Base-class for density calculation errors."""

* Specific exceptions would inherit from these general exceptions.
* Each intermediate exception acts as its own kind of root exception.
* This makes it easier to insulate layers of calling code from API code based on broad functionality.
* This is much better than having all callers catch a long list of very specific Exception subclasses.

### Things to Remember

* Defining root exceotions for your modules allows API consumers to insulate themselves from your API.
* Catching root exceptions can help you find bugs in code that consumes an API.
* Catching the Python Exception base class can help you find bugs in API implementations.
* Intermediate root exceptions let you add more specific types of exceptions in the future without breaking your API consumers.